Here's the code for analyzing listings!

In [ ]:
import pandas
from plotnine import *
import parse
import datetime
import warnings
import math

warnings.filterwarnings('ignore')

In [ ]:
listings = parse.Listings()
df = listings.fetch_data()
df = df[df['Date'] != 'TBA']
df = df[df['Date'] >= datetime.date.today().strftime('%Y-%m-%d')]

listing_df = df

The following code block is going to show you a column plot with the most popular listings.

In [ ]:
(ggplot(listing_df, aes(x='Listing', fill='Listing')) + 
 geom_bar() + 
 theme_bw() +
 theme(axis_text_x=element_text(angle=90, hjust=1), 
       legend_position="none"))

The following code block is going to print top 10 the most listed collections

In [ ]:
listed_df = df.groupby('Collection').size().reset_index(name='counts')
top10 = listed_df.sort_values('counts', ascending=False).head(10)

#add all the information for the top 10 collections from df
top10 = top10.merge(df, on='Collection', how='left').drop_duplicates(subset='Collection', keep='first')
#print top10
for index, row in top10.iterrows():
    print("Collection: " + row['Collection'] + " (Listed " +  str(row['counts']) + ")")
    print("Date: " + row['Date'])
    print("Mint: " + str(row['Price']) + " at " + row['Platform'])
    print("Twitter: " + row['Twitter'] + "\n")

In [ ]:
(ggplot(top10, aes(x='Collection', y='counts', fill='Collection')) +
    geom_bar(stat='identity') +
    theme_bw() +
    theme(axis_text_x=element_text(angle=90, hjust=1),
            legend_position="none") +
    scale_x_discrete(limits=top10.sort_values('counts', ascending=False)['Collection']))

Preparing the dataset with mint dates

In [ ]:
mint_days_df = df
mint_days_df = mint_days_df[mint_days_df.Date != 'TBA']
#Convert Date to datetime
mint_days_df['Date'] = pandas.to_datetime(mint_days_df['Date'])
mint_days_df['Date'] = mint_days_df['Date'].dt.strftime('%Y-%m-%d')

#group by date and count
popular_mint_days = mint_days_df.groupby('Date').size().reset_index(name='counts')
popular_mint_days = popular_mint_days.sort_values('Date', ascending=True)
popular_mint_days = popular_mint_days[popular_mint_days['Date'] >= datetime.datetime.today().strftime('%Y-%m-%d')]

#Plotting the the next 20 popular mint days
(ggplot(popular_mint_days.head(20), aes(x='Date', y='counts', fill='Date')) +
    geom_bar(stat='identity') +
    theme_bw() +
    theme(axis_text_x=element_text(angle=90, hjust=1),
            legend_position="none") +
    scale_x_discrete(limits=popular_mint_days.head(20)['Date']))

In [ ]:
#Mint days by platform
platform_mint_days = mint_days_df.groupby(['Platform', 'Date']).size().reset_index(name='counts')
platform_mint_days = platform_mint_days.sort_values('Date', ascending=True)
platform_mint_days = platform_mint_days[platform_mint_days['Date'] >= datetime.datetime.today().strftime('%Y-%m-%d')]
platform_mint_days = platform_mint_days[platform_mint_days['Platform'] != '']

(ggplot(platform_mint_days.head(20), aes(x='Date', y='counts', fill='Date')) +
    geom_bar(stat='identity') +
    theme_bw() +
    theme(axis_text_x=element_text(angle=90, hjust=1),
            legend_position="none") +
    facet_wrap('~Platform'))

In [ ]:
#Getting the total view of the upcoming mint days
(ggplot(popular_mint_days, aes(x='Date', y='counts', fill='Date')) +
    geom_bar(stat='identity') +
    theme_bw() +
    #remove x axis labels
    theme(axis_text_x=element_blank(),
          legend_position="none") +
    scale_x_discrete(limits=popular_mint_days['Date']))

Проекты с разбивкой по блокчейну.

In [ ]:
# Column plot with number of listings per Platform
df_platform = df.groupby('Platform').size().reset_index(name='counts')
df_platform = df_platform[df_platform['Platform'] != '']
print("Total number of listings: " + str(len(df)))
print(df_platform.sort_values('counts', ascending=False))

In [ ]:
(ggplot(df_platform, aes(x='Platform', y='counts', fill='Platform')) +
    geom_bar(stat='identity') +
    theme_bw() +
    theme(axis_text_x=element_text(angle=90, hjust=1),
            legend_position="none") +
    scale_x_discrete(limits=df_platform.sort_values('counts', ascending=False)['Platform']) +
    geom_text(aes(label='counts'), stat='identity', position=position_stack(vjust=0.5)) + 
    labs(title="Number of listings per Platform", x="Platform", y="Number of listings", fill="Platform"))


Топ-10 по цене проекты.

In [ ]:
top_price_df = df
#clean
top_price_df = top_price_df[top_price_df['Platform'] != '']
top_price_df = top_price_df[top_price_df['Price'] != '']
top_price_df = top_price_df[top_price_df['Price'] != 'TBA']
#Change this accordingly
price_dict = {
    'Ethereum': 1722,
    'Solana': 19,
    'Polygon': 1,
    'Cardano': 0.34,
    'Cronos': 0.069,
    'Elrond': 41.88
}

#iterate through the dataframe and convert the price to USD
for index, row in top_price_df.iterrows():
    try:
        top_price_df.at[index, 'Price'] = float(row['Price'])
    except:
        top_price_df.at[index, 'Price'] = 0
        
top_price_df = top_price_df[top_price_df['Price'] != 0]

#Convert to USD for Price USD column
top_price_df['Price USD'] = top_price_df.apply(lambda row: row['Price'] * price_dict[row['Platform']], axis=1)

print("Top 10 most expensive listings")
top10_price = top_price_df.sort_values('Price USD', ascending=False)

print("Total number of listings: " + str(len(top_price_df)))
print("Median price: $" + str(top_price_df['Price USD'].median()) + " USD\n")

for index, row in top10_price.head(10).iterrows():
    print("Collection: " + row['Collection'])
    print("Date: " + row['Date'])
    print("Mint: $" + str(row['Price']) + " at " + row['Platform'])
    print("Twitter: " + row['Twitter'] + "\n")


In [ ]:
bottom10_price = top_price_df.sort_values('Price USD', ascending=True)

for index, row in bottom10_price.head(10).iterrows():
    print("Collection: " + row['Collection'])
    print("Date: " + row['Date'])
    print("Mint: $" + str(row['Price']) + " at " + row['Platform'])
    print("Twitter: " + row['Twitter'] + "\n")

In [ ]:
def convert_scientific_notation(number):
    return number * 10 ** (int(math.log10(number)) - 5)

#Calculate the total price volume for each platform
platform_price_df = top_price_df.groupby('Platform').sum().reset_index()

print("Total collections with price and platform listed:" + str(len(top_price_df)))
print("Cumulitive total price volume: $" + str(convert_scientific_notation(platform_price_df['Price USD'].sum())) + '\n')

for index, row in platform_price_df.sort_values('Price USD', ascending=False).iterrows():
    print("Platform: " + row['Platform'])
    print("Total Price Volume: $" + str(convert_scientific_notation(row['Price USD'])) + '\n')

In [ ]:
#Plot the total price volume for the next 20 days
next_volume_df = top_price_df

#Convert Date to datetime
next_volume_df['Date'] = pandas.to_datetime(next_volume_df['Date'])
next_volume_df['Date'] = next_volume_df['Date'].dt.strftime('%Y-%m-%d')

#For each date, get the total price volume for each platform
next_volume_df = next_volume_df.groupby(['Date', 'Platform']).sum().reset_index()
next_volume_df = next_volume_df[next_volume_df['Date'] >= datetime.datetime.today().strftime('%Y-%m-%d')]
next_volume_df = next_volume_df.sort_values('Date', ascending=True)

print("Total number of upcoming listings: " + str(len(next_volume_df)))
print(next_volume_df.head(20))

In [ ]:
#Print the total volume for the next 20 days for all platforms
print("Total volume for the next 20 days for all platforms")
print(next_volume_df.groupby('Date').sum().reset_index().head(20))

In [ ]:
#Plot the total price volume for the next 20 days
(ggplot(next_volume_df.head(20), aes(x='Date', y='Price USD')) +
    geom_bar(stat='identity') +
    theme_bw() +
    theme(axis_text_x=element_text(angle=90, hjust=1),
            legend_position="none") +
    scale_x_discrete(limits=next_volume_df.head(20)['Date']) +
    facet_wrap('~Platform', ncol=1) +
    labs(title="Total price volume for the next 20 days", x="Date", y="Total price volume", fill="Platform"))